# Airport Decision Architecture & Transformation Analysis

This notebook supports the flagship **Business Architecture & Decision Intelligence** narrative by providing structured analytical evidence.

The purpose is not operational optimisation, but to identify **high-leverage decision domains** within airport systems under regulatory, capacity, sustainability, and capital constraints.


## Strategic question

Which airport decision domains create disproportionate value or risk, and how should airport leadership prioritise data, analytics, and transformation investment?


## Dataset overview

The dataset models airport decision domains across different airport archetypes, capturing:

- Financial exposure
- Customer and airline experience impact
- Regulatory constraint
- Data and automation readiness
- Sustainability dependency
- Cross-stakeholder complexity

The intent is **comparative prioritisation**, not forecasting.


In [ ]:
import pandas as pd

df = pd.read_csv("../data/airport_decision_architecture_dataset.csv")
df.head()

## Decision domain aggregation

To support architecture-level prioritisation, decisions are aggregated by decision domain rather than airport.


In [ ]:
summary = df.groupby("decision_domain").agg({
    "annual_cost_exposure_gbp_m": "mean",
    "customer_experience_impact": "mean",
    "automation_feasibility": "mean",
    "regulatory_constraint_level": "mean",
    "sustainability_dependency": "mean",
    "cross_stakeholder_dependency": "mean"
}).reset_index()

summary

## Decision leverage scoring

A composite leverage score is used to highlight decision domains where:

- Financial exposure is high
- Customer and airline impact is high
- Automation potential exists
- Regulatory complexity is material

This does not produce a 'winner', but a prioritisation lens.


In [ ]:
summary["decision_leverage_score"] = (
    summary["annual_cost_exposure_gbp_m"].rank(pct=True) +
    summary["customer_experience_impact"].rank(pct=True) +
    summary["automation_feasibility"].rank(pct=True) +
    summary["regulatory_constraint_level"].rank(pct=True)
)

summary.sort_values("decision_leverage_score", ascending=False)

## Architecture implications

Decision domains with high leverage scores typically:

- Require cross-functional and cross-stakeholder governance
- Cannot be optimised locally without system-wide consequences
- Benefit from architecture-led sequencing rather than isolated analytics initiatives


## Key observations

- A small subset of decision domains dominate financial and experience outcomes
- Regulatory and sustainability dependencies amplify decision risk
- Data maturity varies significantly by decision domain, not airport size
- Transformation impact is driven by **decision sequencing**, not tool deployment
